# Classificação de imagem - Faces do Nilson vs Outros

### Projeto final da DIO - Trilha da EY

Esse notebook visa realizar um treinamento para ser possível identificar a face de uma determinada pessoa (no caso eu - Nilson) em detrimento à face de qualquer outra pessoa. 

Foram utilizados os conhecimentos adquiridos na trilha EY Fast Track Specialist - Machine Learning da DIO em parceria com a EY. 

# Importação dos pacotes

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [4]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

# Data Loading

Aqui serão baixados os datasets que estão armazenados no meu Google Drive.

Dentro existem as classes "outros" e "nilson" que serão utilizadas no treinamento e na classificação.

In [5]:
_URL = 'https://drive.google.com/u/0/uc?id=1GOCmH4-sA3oHNP4pVtyHJ3zXzPliI23m&export=download&confirm=t&uuid=4a487169-dbf6-4ba2-81a3-7db372d2993e&at=ALAFpqw9j5keI4N25VLuX4YWZfxu:1667081119859'
zip_dir = tf.keras.utils.get_file('dataset-classific-nilson.zip', origin=_URL, extract=True)

250695841/250695841 [==============================] - 2s 0us/step


O dataset que será baixado seguirá a seguinte estrutura de diretórios: 

<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>dataset-classific-nilson</b>
|__ <b>train</b>
    |______ <b>outros</b>
    |______ <b>nilson</b>
|__ <b>validation</b>
    |______ <b>outros</b>
    |______ <b>nilson</b>
</pre>

In [6]:
zip_dir_base = os.path.dirname(zip_dir)
!find $zip_dir_base -type d -print

/root/.keras/datasets
/root/.keras/datasets/dataset-classfic-nilson
/root/.keras/datasets/dataset-classfic-nilson/train
/root/.keras/datasets/dataset-classfic-nilson/train/outros
/root/.keras/datasets/dataset-classfic-nilson/train/nilson
/root/.keras/datasets/dataset-classfic-nilson/validation
/root/.keras/datasets/dataset-classfic-nilson/validation/outros
/root/.keras/datasets/dataset-classfic-nilson/validation/nilson


In [7]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'dataset-classfic-nilson')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_outros_dir = os.path.join(train_dir, 'outros')  # diretório com as imagens de treinamento da classe "outros"
train_nilson_dir = os.path.join(train_dir, 'nilson')  # diretório com as imagens de treinamento da classe "nilson"
validation_outros_dir = os.path.join(validation_dir, 'outros')  # diretório com as imagens de validação da classe "outros"
validation_nilson_dir = os.path.join(validation_dir, 'nilson')  # diretório com as imagens de validação da classe "nilson"

In [8]:
num_outros_tr = len(os.listdir(train_outros_dir))
num_nilson_tr = len(os.listdir(train_nilson_dir))

num_outros_val = len(os.listdir(validation_outros_dir))
num_nilson_val = len(os.listdir(validation_nilson_dir))

total_train = num_outros_tr + num_nilson_tr
total_val = num_outros_val + num_nilson_val

In [9]:
print('total de imagens de treinamento "outros":', num_outros_tr)
print('total de imagens de treinamento "nilson"', num_nilson_tr)

print('total de imagens de validação "outros"', num_outros_val)
print('total de imagens de validação "nilson"', num_nilson_val)
print("--")
print("Total de imagens de treinamento:", total_train)
print("Total de imagens de validação:", total_val)

total de imagens de treinamento "outros": 80
total de imagens de treinamento "nilson" 80
total de imagens de validação "outros" 20
total de imagens de validação "nilson" 20
--
Total de imagens de treinamento: 160
Total de imagens de validação: 40


#Parâmetros do modelo

In [48]:
BATCH_SIZE = 10  
IMG_SHAPE  = 300  #Tamanho das imagens em pixels

# Preparação dos dados

In [49]:
train_image_generator      = ImageDataGenerator(rescale=1./255)  # Generator dos dados de treinamento
validation_image_generator = ImageDataGenerator(rescale=1./255)  # Generator dos dados de validação

In [50]:
train_data_gen = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                           seed=123,
                                                           image_size=(IMG_SHAPE,IMG_SHAPE), #(300,300)
                                                           batch_size=BATCH_SIZE)

Found 160 files belonging to 2 classes.


In [51]:
class_names = train_data_gen.class_names  #Cria uma lista com as classes que serão utilizadas para posterior classificação
print(class_names)

['nilson', 'outros']


In [52]:
val_data_gen = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                           seed=123,
                                                           image_size=(IMG_SHAPE,IMG_SHAPE), #(300,300)
                                                           batch_size=BATCH_SIZE)

Found 40 files belonging to 2 classes.


# Criação do modelo

## Definição do modelo

Aqui serão definidos os algoritmos do modelo.
Serão mantidos os mesmos passados no exemplo pelo prof. Diego Bruno. 
Para a predição e classificação da imagem entre as duas classes - nilson e outros -  será utilizada `softmax`


In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2)
])

### Compilação do modelo


In [53]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Sumário do modelo

Visualizando todas as layers da nossa rede.

In [54]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 128)       147584    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 128)      0

### Treinamento do modelo

Serão utizadas 50 épocas - número interessante para obter uma boa acurácia neste experimento

In [ ]:
EPOCHS = 50
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE)))
)

### Visualização do resultado do treinamento

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Acurácia do treinamento')
plt.plot(epochs_range, val_acc, label='Acurácia da validação')
plt.legend(loc='lower right')
plt.title('Acurácia do treinamento e validação')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Perda no treinamento')
plt.plot(epochs_range, val_loss, label='Perda na validação')
plt.legend(loc='upper right')
plt.title('Perda no treinamento e validação')
plt.savefig('./foo.png')
plt.show()

In [57]:
import cv2
from google.colab.patches import cv2_imshow

In [64]:
img_array = None    #Zera as variáveis apenas para garantia de valores novos
image = None
resize = None
prediction = None
score = None

image = cv2.imread('/content/sample_data/nil1.jpg')   #Carrega a imagem para predição

In [65]:
resize = cv2.resize(image,(300,300),interpolation = cv2.INTER_AREA)    #Redimensiona a imagem carregada conforme os padrões do modelo

In [66]:
img_array = tf.keras.utils.img_to_array(resize)  #Transforma a imagem em array para ser predita
img_array = tf.expand_dims(img_array,0)

In [ ]:
prediction = model.predict(img_array)
score = tf.nn.softmax(prediction[0])

In [ ]:
if class_names[np.argmax(score)] == 'nilson':
  print('DISPOSITIVO DESBLOQUADO - NILSON IDENTIFICADO COM ' + str(100 * np.max(score)) + ' % DE CERTEZA.' )
else:
  print('ERRO - O DISPOSITIVO NÃO PODE SER DESBLOQUEADO PORQUE A FACE IDENTIFICADA NÃO É A DO NILSON (' + str(100 * np.max(score)) + ' % DE CERTEZA).' )